# BIOINF 575 - Group Project - Kegg Pathway Overlap
#### Authors:
* Ryan Rebernick
* Elysia Chou
* Mahnoor Gondal
* Tusharika Rastogi


## README
This is the space where I (Elysia) clean and merge the code.

In [2]:
import numpy as np
import pandas as pd

In [3]:
# Create a path string to adjust at the beginning, depending on whose machine the code is running.
path = '../../int/'

## 1. Map/merge the information by PATHWAY_ID and GENE_ID. 

#### First we read in the data and store in dataframes for easy access.

In [ ]:

# Read the pathway names
pathway_pinfo = pd.read_table('http://rest.kegg.jp/list/pathway/hsa', header=None)
pathway_pinfo.columns = ['PATHWAY_ID', 'PATHWAY_INFO']
print(pathway_pinfo.head())
print(np.shape(pathway_pinfo))

# Read the gene-pathway
geneid_pathway = pd.read_table('http://rest.kegg.jp/link/pathway/hsa', header=None)
geneid_pathway.columns = ['GENE_ID', 'PATHWAY_ID']
print(geneid_pathway.head())
print(np.shape(geneid_pathway))

# Read the pathway names
geneid_ginfo = pd.read_table('http://rest.kegg.jp/list/hsa', header=None)
geneid_ginfo.columns = ['GENE_ID', 'GENE_INFO']
print(geneid_ginfo.head())
print(np.shape(geneid_ginfo))


#### Next we need to merge the information by pathway ids and gene ids

UPDATE: The gene IDs with NaN gene info are not present in the original KEGG file.

Note that in merging, some genes did not have values in the geneid_ginfo file. These were left as NaN. I have shown them for reference.

In [ ]:

# combine the pathway information and the gene ids
merged = pd.merge(pathway_pinfo, geneid_pathway, on = 'PATHWAY_ID', how = 'outer')

# add the gene info
merged = pd.merge(merged, geneid_ginfo, on = 'GENE_ID', how = 'left')

# sort and look at the result
merged = merged.sort_values(by = 'PATHWAY_ID')
#print(merged.head())
#print(np.shape(merged))

# View the NA genes
print("Gene info with NaN values:")
print(merged[merged['GENE_INFO'].isnull()])



#### Save this file for future use

In [ ]:

merged.to_csv(path + '2.1_merged_genes_pathways.csv', sep = ',', mode = 'w', index=False)
    

## 2. Compute the number of overlapping genes between every 2 pathways.

Our goal is to create a dataframe with 4 columns: pathway 1, pathway 2, the number of overlapping genes, and which genes overlap.
To do this we will:

* create a dictionary of each pathway to a set of its genes.
* Then we will loop through each and compare it to the remaining pathways.
* The output of this will be exported as a .csv file

#### First we read in our data that we previously generated
If running separate scripts or picking up where you left off.

In [8]:
# Run this is you are starting from this step
merged = pd.read_table(path + '2.1_merged_genes_pathways.csv', sep=',')
merged = merged.rename(columns={'pathway_id': 'PATHWAY_ID', 'pathway_info': 'PATHWAY_INFO', 'gene_id': 'GENE_ID', 'gene_info': 'GENE_INFO'})
# merged.head()

#### Next we will create dictionaries that store the pathways and a set of each pathways respective genes.


In [9]:

# all the unique pathways
unique_pathways = np.unique(merged['PATHWAY_ID'])
print(unique_pathways[:2])

# dictionary to store each pathwyas genes
pathway_genes = {}

# get the genes for each pathway and store in dictionary
for pw in unique_pathways:
    
    # rows of df corresponding to current pathway
    cur_pw = merged[merged.PATHWAY_ID == pw]
    # the genes from the current rows
    cur_genes = set(cur_pw.GENE_ID)
    # add to dictionary
    pathway_genes.update({pw:cur_genes})

# look at the first dicionary entry
print('\n')
print(list(pathway_genes.items())[:1])
print()

['path:hsa00010' 'path:hsa00020']


[('path:hsa00010', {'hsa:55276', 'hsa:5315', 'hsa:5236', 'hsa:5230', 'hsa:501', 'hsa:84532', 'hsa:2023', 'hsa:5211', 'hsa:1738', 'hsa:5105', 'hsa:3101', 'hsa:1737', 'hsa:226', 'hsa:3939', 'hsa:3945', 'hsa:3948', 'hsa:55902', 'hsa:2026', 'hsa:5232', 'hsa:2538', 'hsa:224', 'hsa:92483', 'hsa:218', 'hsa:10327', 'hsa:125', 'hsa:127', 'hsa:5213', 'hsa:26330', 'hsa:5223', 'hsa:230', 'hsa:130589', 'hsa:160287', 'hsa:441531', 'hsa:5106', 'hsa:2027', 'hsa:229', 'hsa:130', 'hsa:5313', 'hsa:2203', 'hsa:128', 'hsa:669', 'hsa:83440', 'hsa:80201', 'hsa:124', 'hsa:7167', 'hsa:217', 'hsa:2821', 'hsa:3098', 'hsa:387712', 'hsa:57818', 'hsa:5162', 'hsa:5160', 'hsa:2597', 'hsa:92579', 'hsa:126', 'hsa:3099', 'hsa:8789', 'hsa:131', 'hsa:222', 'hsa:219', 'hsa:2645', 'hsa:221', 'hsa:5161', 'hsa:5214', 'hsa:5224', 'hsa:223', 'hsa:9562'})]



#### Next we will loop through each pathway and crossref the overlapping genes with all the other pathways. The number of crossovers will be stored in a dataframe.

In [10]:

# lists to store values; will be converted into dataframe at the end
pw_1_list = list()
pw_2_list = list()
gene_overlap_list = list()
number_overlap_list = list()

# for each pathway in the unique pathways
for n in range(0, len(unique_pathways)):
    
    # first pathway
    pw_1 = unique_pathways[n]
    # genes from first pathway
    pw_1_genes = pathway_genes.get(pw_1)
    
    # loop through other pathway combinations,
    # such that no pair of pathways shows up twice,
    # nor do we take pairs of which each element is the same pathway.
    for nn in range(n+1, len(unique_pathways)):
        
        # second pathway
        pw_2 = unique_pathways[nn]
        # genes from first pathway
        pw_2_genes = pathway_genes.get(pw_2)
    
        # compute overlapping genes
        overlaps = pw_1_genes.intersection(pw_2_genes)
        num_overalps = len(overlaps)
    
        # add to lists
        pw_1_list.append(pw_1)
        pw_2_list.append(pw_2)
        gene_overlap_list.append(overlaps)
        number_overlap_list.append(num_overalps)

        
# create dataframe from lists
pathway_gene_overlaps = pd.DataFrame(list(zip(pw_1_list, pw_2_list, number_overlap_list, gene_overlap_list)), 
                                     columns = ['PATHWAY_ID1', 'PATHWAY_ID2', 'NUM_OVERLAPPING_GENES', 'OVERLAPPING_GENES'])

# look at the first entries
print(np.shape(pathway_gene_overlaps))
pathway_gene_overlaps[:10]

(59340, 4)


,PATHWAY_ID1,PATHWAY_ID2,NUM_OVERLAPPING_GENES,OVERLAPPING_GENES
0,path:hsa00010,path:hsa00020,7,"{hsa:5162, hsa:5160, hsa:1737, hsa:1738, hsa:5..."
1,path:hsa00010,path:hsa00030,11,"{hsa:55276, hsa:5236, hsa:2203, hsa:8789, hsa:..."
2,path:hsa00010,path:hsa00040,1,{hsa:10327}
3,path:hsa00010,path:hsa00051,13,"{hsa:2203, hsa:3099, hsa:8789, hsa:80201, hsa:..."
4,path:hsa00010,path:hsa00052,14,"{hsa:55276, hsa:92579, hsa:5236, hsa:2538, hsa..."
5,path:hsa00010,path:hsa00053,6,"{hsa:224, hsa:501, hsa:219, hsa:10327, hsa:217..."
6,path:hsa00010,path:hsa00061,0,{}
7,path:hsa00010,path:hsa00062,0,{}
8,path:hsa00010,path:hsa00071,12,"{hsa:130, hsa:126, hsa:224, hsa:501, hsa:128, ..."
9,path:hsa00010,path:hsa00100,0,{}


#### Save this file for future use

In [ ]:
pathway_gene_overlaps.to_csv(path + '2.2_overlapping_genes.csv', sep = ',', mode = 'w', index=False)

## 3. Save the result to a file KEGG_crosstalk.csv 
This file should be saved with the following columns: PATHWAY_ID1, PATHWAY_NAME1, PATHWAY_ID2, PATHWAY_NAME2. Order the results descending by the number of overlapping genes where PATHWAY_ID1 is different than PATHWAY_ID2.

In [32]:
# Run if starting from here
# pathway_pinfo = merged[['PATHWAY_ID', 'PATHWAY_INFO']]
# pathway_pinfo = pathway_info.drop_duplicates()

In [18]:
#Add pathway information for pathway ID1
# Using original pathway_pinfo obtained from KEGG (see step 1)
Combined_pathways1 = pd.merge(pathway_gene_overlaps,
                 pathway_pinfo,
                 left_on='PATHWAY_ID1',
                 right_on='PATHWAY_ID')

# look at the first entries
print(np.shape(Combined_pathways1))
# DELETE LATER Combined_pathways1[346:700]

(59340, 6)


In [20]:
#Add pathway information for pathway ID2 and rearranging the dataframe into a new dataframe with updated information
Combined_pathways2 = pd.merge(Combined_pathways1,
                 pathway_pinfo,
                 left_on = "PATHWAY_ID2",
                 right_on = "PATHWAY_ID" )
#print(Combined_pathways2.head())

In [ ]:
# DELETE LATER
print(Combined_pathways2.head())

In [21]:
del Combined_pathways2["PATHWAY_ID_x"],Combined_pathways2["PATHWAY_ID_y"]
Combined_pathways2 = Combined_pathways2.rename(columns={'PATHWAY_INFO_x': 'PATHWAY_NAME1', 'PATHWAY_INFO_y': 'PATHWAY_NAME2'})
Combined_pathways2 = Combined_pathways2[["PATHWAY_ID1", "PATHWAY_NAME1", "PATHWAY_ID2", "PATHWAY_NAME2", "NUM_OVERLAPPING_GENES", "OVERLAPPING_GENES"]]
Updated_data = Combined_pathways2.sort_values(by=['NUM_OVERLAPPING_GENES'], ascending=False)

# look at the first entries
print(np.shape(Updated_data))
Updated_data[:10]

(59340, 6)


,PATHWAY_ID1,PATHWAY_NAME1,PATHWAY_ID2,PATHWAY_NAME2,NUM_OVERLAPPING_GENES,OVERLAPPING_GENES
35772,path:hsa05010,Alzheimer disease - Homo sapiens (human),path:hsa05022,Pathways of neurodegeneration - multiple disea...,333,"{hsa:4217, hsa:6391, hsa:7419, hsa:776, hsa:14..."
35774,path:hsa05014,Amyotrophic lateral sclerosis - Homo sapiens (...,path:hsa05022,Pathways of neurodegeneration - multiple disea...,294,"{hsa:4217, hsa:55860, hsa:440567, hsa:5861, hs..."
35775,path:hsa05016,Huntington disease - Homo sapiens (human),path:hsa05022,Pathways of neurodegeneration - multiple disea...,251,"{hsa:774, hsa:55860, hsa:4217, hsa:440567, hsa..."
35773,path:hsa05012,Parkinson disease - Homo sapiens (human),path:hsa05022,Pathways of neurodegeneration - multiple disea...,228,"{hsa:4217, hsa:7318, hsa:440567, hsa:7345, hsa..."
34979,path:hsa05014,Amyotrophic lateral sclerosis - Homo sapiens (...,path:hsa05016,Huntington disease - Homo sapiens (human),224,"{hsa:4217, hsa:55860, hsa:440567, hsa:56171, h..."
35777,path:hsa05020,Prion disease - Homo sapiens (human),path:hsa05022,Pathways of neurodegeneration - multiple disea...,220,"{hsa:774, hsa:440567, hsa:6391, hsa:5534, hsa:..."
35506,path:hsa05010,Alzheimer disease - Homo sapiens (human),path:hsa05020,Prion disease - Homo sapiens (human),214,"{hsa:440567, hsa:6391, hsa:5534, hsa:7419, hsa..."
34452,path:hsa05010,Alzheimer disease - Homo sapiens (human),path:hsa05012,Parkinson disease - Homo sapiens (human),208,"{hsa:4217, hsa:440567, hsa:23516, hsa:6391, hs..."
34714,path:hsa05010,Alzheimer disease - Homo sapiens (human),path:hsa05014,Amyotrophic lateral sclerosis - Homo sapiens (...,206,"{hsa:4217, hsa:440567, hsa:6391, hsa:22863, hs..."
34977,path:hsa05010,Alzheimer disease - Homo sapiens (human),path:hsa05016,Huntington disease - Homo sapiens (human),202,"{hsa:4217, hsa:440567, hsa:6391, hsa:22863, hs..."


In [37]:
#Save the result to a file KEGG_crosstalk.csv with the following columns: 
#PATHWAY_ID1, PATHWAY_NAME1, PATHWAY_ID2, PATHWAY_NAME2. 

Updated_data.to_csv(path + '2.3_KEGG_crosstalk.csv')

## 4. Compute a rank of the genes based on how many pathways they appear on and save it to a file.

In [24]:
#Function to extract gene_description
def get_description(info):
    if type(info) == float:
        return ""
    return info.split(";")[-1]
    

In [26]:
#Apply function to extract gene description and add it to the data
merged['gene_description'] = merged['GENE_INFO'].apply(get_description)

In [27]:
#Apply gene_id for genes without any pathway assigned i.e NaN in description
merged.gene_description[(merged.gene_description == "")] = merged.GENE_ID[(merged.gene_description == "")]

# look at the first entries
print(np.shape(merged))
merged[:10]

(35381, 5)


,PATHWAY_ID,PATHWAY_INFO,GENE_ID,GENE_INFO,gene_description
0,path:hsa00010,Glycolysis / Gluconeogenesis - Homo sapiens (h...,hsa:10327,"AKR1A1, ALDR1, ALR, ARM, DD3, HEL-S-6; aldo-ke...",aldo-keto reductase family 1 member A1
1,path:hsa00010,Glycolysis / Gluconeogenesis - Homo sapiens (h...,hsa:3945,"LDHB, HEL-S-281, LDH-B, LDH-H, LDHBD, TRG-5; l...",lactate dehydrogenase B
2,path:hsa00010,Glycolysis / Gluconeogenesis - Homo sapiens (h...,hsa:3948,"LDHC, CT32, LDH3, LDHX; lactate dehydrogenase C",lactate dehydrogenase C
3,path:hsa00010,Glycolysis / Gluconeogenesis - Homo sapiens (h...,hsa:441531,"PGAM4, PGAM-B, PGAM1, PGAM3, dJ1000K24.1; phos...",phosphoglycerate mutase family member 4
4,path:hsa00010,Glycolysis / Gluconeogenesis - Homo sapiens (h...,hsa:501,"ALDH7A1, ATQ1, EPD, PDE; aldehyde dehydrogenas...",aldehyde dehydrogenase 7 family member A1
5,path:hsa00010,Glycolysis / Gluconeogenesis - Homo sapiens (h...,hsa:5106,"PCK2, PEPCK, PEPCK-M, PEPCK2; phosphoenolpyruv...","phosphoenolpyruvate carboxykinase 2, mitochon..."
6,path:hsa00010,Glycolysis / Gluconeogenesis - Homo sapiens (h...,hsa:5160,"PDHA1, PDHA, PDHAD, PDHCE1A, PHE1A; pyruvate d...",pyruvate dehydrogenase E1 subunit alpha 1
7,path:hsa00010,Glycolysis / Gluconeogenesis - Homo sapiens (h...,hsa:5161,"PDHA2, PDHAL; pyruvate dehydrogenase E1 subuni...",pyruvate dehydrogenase E1 subunit alpha 2
8,path:hsa00010,Glycolysis / Gluconeogenesis - Homo sapiens (h...,hsa:5162,"PDHB, PDHBD, PDHE1-B, PDHE1B, PHE1B; pyruvate ...",pyruvate dehydrogenase E1 subunit beta
9,path:hsa00010,Glycolysis / Gluconeogenesis - Homo sapiens (h...,hsa:5211,"PFKL, ATP-PFK, PFK-B, PFK-L; phosphofructokina...","phosphofructokinase, liver type"


In [28]:
#Compute rank of the genes based on how many pathways they appear in
gene_rank = merged.groupby(['gene_description'])['PATHWAY_ID'].count().sort_values(ascending=False)
gene_rank = pd.DataFrame(gene_rank)
gene_rank['gene_rank'] = gene_rank['PATHWAY_ID']

del gene_rank['PATHWAY_ID']

# look at the first entries
print(np.shape(gene_rank))
gene_rank[:10]

(8102, 1)


,gene_rank
gene_description,
mitogen-activated protein kinase 1,117
mitogen-activated protein kinase 3,117
"phosphatidylinositol-4,5-bisphosphate 3-kinase catalytic subunit alpha",106
"phosphatidylinositol-4,5-bisphosphate 3-kinase catalytic subunit beta",106
"phosphatidylinositol-4,5-bisphosphate 3-kinase catalytic subunit delta",106
phosphoinositide-3-kinase regulatory subunit 1,103
phosphoinositide-3-kinase regulatory subunit 3,103
phosphoinositide-3-kinase regulatory subunit 2,103
AKT serine/threonine kinase 1,100


In [36]:
#Save the result to a file
gene_rank.to_csv(path + '2.4_Gene_Rank.csv')

## 5. Retrieve a set of the pathways the top 3 genes appear on.

In [ ]:
# Run this if you start from this step:
# gene_rank = pd.read_table(path + '2.4_Gene_Rank.csv', sep=',')

In [ ]:
# gene_rank.index.apply(strip())

In [ ]:
# merged.gene_description.apply(str.strip())

In [29]:
# Top n genes to extract
n = 3
top_genes = gene_rank.iloc[:n,]
# The 3rd gene might change depending on the way the gene_rank dataframe was sorted.
top_genes

,gene_rank
gene_description,
mitogen-activated protein kinase 1,117
mitogen-activated protein kinase 3,117
"phosphatidylinositol-4,5-bisphosphate 3-kinase catalytic subunit alpha",106


In [30]:
# We verified that the gene description is unique to the gene ID, at least in the case of our top n genes
top_genes_pinfo = merged[merged.gene_description.isin(top_genes.index)]
top_genes_pinfo

,PATHWAY_ID,PATHWAY_INFO,GENE_ID,GENE_INFO,gene_description
1851,path:hsa00562,Inositol phosphate metabolism - Homo sapiens (...,hsa:5290,"PIK3CA, CLAPO, CLOVE, CWS5, MCAP, MCM, MCMTC, ...","phosphatidylinositol-4,5-bisphosphate 3-kinas..."
3202,path:hsa01100,Metabolic pathways - Homo sapiens (human),hsa:5290,"PIK3CA, CLAPO, CLOVE, CWS5, MCAP, MCM, MCMTC, ...","phosphatidylinositol-4,5-bisphosphate 3-kinas..."
5006,path:hsa01521,EGFR tyrosine kinase inhibitor resistance - Ho...,hsa:5595,"MAPK3, ERK-1, ERK1, ERT2, HS44KDAP, HUMKER1A, ...",mitogen-activated protein kinase 3
5007,path:hsa01521,EGFR tyrosine kinase inhibitor resistance - Ho...,hsa:5594,"MAPK1, ERK, ERK-2, ERK2, ERT1, MAPK2, NS13, P4...",mitogen-activated protein kinase 1
5020,path:hsa01521,EGFR tyrosine kinase inhibitor resistance - Ho...,hsa:5290,"PIK3CA, CLAPO, CLOVE, CWS5, MCAP, MCM, MCMTC, ...","phosphatidylinositol-4,5-bisphosphate 3-kinas..."
...,...,...,...,...,...
34767,path:hsa05415,Diabetic cardiomyopathy - Homo sapiens (human),hsa:5290,"PIK3CA, CLAPO, CLOVE, CWS5, MCAP, MCM, MCMTC, ...","phosphatidylinositol-4,5-bisphosphate 3-kinas..."
35050,path:hsa05417,Lipid and atherosclerosis - Homo sapiens (human),hsa:5595,"MAPK3, ERK-1, ERK1, ERT2, HS44KDAP, HUMKER1A, ...",mitogen-activated protein kinase 3
35058,path:hsa05417,Lipid and atherosclerosis - Homo sapiens (human),hsa:5290,"PIK3CA, CLAPO, CLOVE, CWS5, MCAP, MCM, MCMTC, ...","phosphatidylinositol-4,5-bisphosphate 3-kinas..."
35064,path:hsa05417,Lipid and atherosclerosis - Homo sapiens (human),hsa:5594,"MAPK1, ERK, ERK-2, ERK2, ERT1, MAPK2, NS13, P4...",mitogen-activated protein kinase 1


In [34]:
# Create a dictionary with n elements,
# where the keys are the gene IDs and the values are the sets of pathways.
dict_geneid_pw = dict()
top_gene_ids = top_genes_pinfo.GENE_ID.unique()
print(f'Top {n} ranked genes:')
for gene in top_gene_ids:
    # Could use merged dataframe or geneid_pathway dataframe from source
    dict_geneid_pw[gene] = set(top_genes_pinfo[top_genes_pinfo.GENE_ID == gene].PATHWAY_ID)
    # For description purposes
    g_description = top_genes_pinfo[top_genes_pinfo['GENE_ID'] == gene].gene_description.unique()[0]
    print(f'{gene} --{g_description}: {len(dict_geneid_pw[gene])} pathways')
    print(f'including but not limited to {list(dict_geneid_pw[gene])[:10]}\n')

    

Top 3 ranked genes:
hsa:5290 -- phosphatidylinositol-4,5-bisphosphate 3-kinase catalytic subunit alpha: 106 pathways
including but not limited to ['path:hsa05222', 'path:hsa05160', 'path:hsa05415', 'path:hsa04915', 'path:hsa04213', 'path:hsa04960', 'path:hsa04810', 'path:hsa00562', 'path:hsa04660', 'path:hsa05417']

hsa:5595 -- mitogen-activated protein kinase 3: 117 pathways
including but not limited to ['path:hsa04270', 'path:hsa05160', 'path:hsa04915', 'path:hsa04657', 'path:hsa04350', 'path:hsa04960', 'path:hsa04810', 'path:hsa04660', 'path:hsa05417', 'path:hsa04540']

hsa:5594 -- mitogen-activated protein kinase 1: 117 pathways
including but not limited to ['path:hsa04270', 'path:hsa05160', 'path:hsa04915', 'path:hsa04657', 'path:hsa04350', 'path:hsa04960', 'path:hsa04810', 'path:hsa04660', 'path:hsa05417', 'path:hsa04540']



In [35]:
# Save to tab-delimited file
with open(path + '2.5_top_genes_pathways.txt', 'w') as file5:
    for key in dict_geneid_pw.keys():
        file5.write(key)
        file5.write('\t')
        file5.write(str(dict_geneid_pw[key]))
        file5.write('\n')
    

## 6. Compute and display a Venn diagram for number of overlapping pathways for the top 3 genes.